In [ ]:
# import os
# import mlflow
# import mlflow.pytorch
# import dagshub
# from ultralytics import YOLO

# # ====================================
# # CONFIG
# # ====================================

# REPO_OWNER = "sarthaksaitwal3"
# REPO_NAME = "final-road-detect-defect"
# EXPERIMENT_NAME = "YOLO_model"
# BEST_MODEL_PATH = "D:\Projects\Final-Road-Detect-Defect\models/best.pt"

# # ====================================
# # AUTH
# # ====================================

# os.environ["MLFLOW_TRACKING_USERNAME"] = "sarthaksaitwal3"
# os.environ["MLFLOW_TRACKING_PASSWORD"] = "13f873a3c905b8da96417985e56aef19eaf2d7c6"

# # ====================================
# # CONNECT
# # ====================================

# dagshub.init(
#     repo_owner=REPO_OWNER,
#     repo_name=REPO_NAME,
#     mlflow=True,
# )

# mlflow.set_experiment(EXPERIMENT_NAME)

# print("✅ Connected to DagsHub")

# # ====================================
# # LOAD YOLO MODEL
# # ====================================

# yolo_model = YOLO(BEST_MODEL_PATH)
# torch_model = yolo_model.model  # Extract underlying PyTorch model

# print("✅ YOLO model loaded")

# # ====================================
# # LOG EVERYTHING
# # ====================================

# with mlflow.start_run(run_name="baseline_model_v1") as run:

#     # ---- Log Parameters ----
#     mlflow.log_param("architecture", "YOLOv8s")
#     mlflow.log_param("task", "object_detection")
#     mlflow.log_param("num_classes", 3)

#     # ---- 1️⃣ Log raw best.pt as artifact ----
#     mlflow.log_artifact(BEST_MODEL_PATH, artifact_path="artifacts")

#     # ---- 2️⃣ Log as MLflow PyTorch Model (Flavor) ----
#     mlflow.pytorch.log_model(
#         pytorch_model=torch_model,
#         name="model"
#     )

#     print("✅ Model + artifacts logged successfully")
#     print("Run ID:", run.info.run_id)

# print("\nGo to DagsHub → Experiments → YOLO_model → This Run")
# print("You will see:")
# print("• Artifacts → best.pt")
# print("• Models → model")
# print("You can now register manually.")

Initialized MLflow to track repo "sarthaksaitwal3/final-road-detect-defect"

Repository sarthaksaitwal3/final-road-detect-defect initialized!

✅ Connected to DagsHub
✅ YOLO model loaded


2026/02/22 22:09:01 WARNING mlflow.pytorch: Saving pytorch model by Pickle or CloudPickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is to set 'export_model' to True to save the pytorch model using the safe graph model format.


✅ Model + artifacts logged successfully
Run ID: 204d78d888c847cd88bddde95be0c82b
🏃 View run baseline_model_v1 at: https://dagshub.com/sarthaksaitwal3/final-road-detect-defect.mlflow/#/experiments/1/runs/204d78d888c847cd88bddde95be0c82b
🧪 View experiment at: https://dagshub.com/sarthaksaitwal3/final-road-detect-defect.mlflow/#/experiments/1

Go to DagsHub → Experiments → YOLO_model → This Run
You will see:
• Artifacts → best.pt
• Models → model
You can now register manually.


In [7]:
import os
print(os.getcwd())

d:\Projects\Final-Road-Detect-Defect\notebooks


In [ ]:
import os
import sys
import torch
import subprocess
from ultralytics import YOLO
import mlflow
import mlflow.pytorch
import dagshub
from dotenv import load_dotenv

# ======================================================
# DETECT ENVIRONMENT
# ======================================================

IN_KAGGLE = os.path.exists('/kaggle/working')
IN_COLAB = False

if not IN_KAGGLE:
    try:
        import google.colab
        IN_COLAB = True
    except Exception:
        pass

# ======================================================
# LOAD ENVIRONMENT VARIABLES
# ======================================================

if IN_KAGGLE or IN_COLAB:
    MLFLOW_TRACKING_USERNAME = "sarthaksaitwal3"
    MLFLOW_TRACKING_PASSWORD = "13f873a3c905b8da96417985e56aef19eaf2d7c6"
else:
    load_dotenv()
    MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
    MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

os.environ["MLFLOW_TRACKING_USERNAME"] = MLFLOW_TRACKING_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = MLFLOW_TRACKING_PASSWORD

# ======================================================
# GPU INFO & PATH CONFIGURATION
# ======================================================

print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    device = 0
    
    if IN_KAGGLE:
        PROJECT_DIR = "/kaggle/input/road-defect-dataset"
        OUTPUT_DIR = "/kaggle/working"
        print("📍 Running in Kaggle")
    elif IN_COLAB:
        PROJECT_DIR = "/content/drive/MyDrive/WIT-Hackthon"
        OUTPUT_DIR = f"{PROJECT_DIR}/outputs/models"
        print("📍 Running in Google Colab")
    else:
        PROJECT_DIR = "d:/Projects/Final-Road-Detect-Defect"
        OUTPUT_DIR = f"{PROJECT_DIR}/outputs/models"
        print("📍 Running locally with GPU")
else:
    print("⚠️  No GPU detected. Using CPU.")
    device = "cpu"
    
    if IN_KAGGLE:
        PROJECT_DIR = "/kaggle/input/datasets/sarthaksaitwal/road-defect-dataset"
        OUTPUT_DIR = "/kaggle/working"
    elif IN_COLAB:
        PROJECT_DIR = "/content/drive/MyDrive/WIT-Hackthon"
        OUTPUT_DIR = f"{PROJECT_DIR}/outputs/models"
    else:
        PROJECT_DIR = "d:/Projects/Final-Road-Detect-Defect"
        OUTPUT_DIR = f"{PROJECT_DIR}/outputs/models"

print(f"Using device: {device}")
print(f"Project directory: {PROJECT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

# ======================================================
# CONFIGURATION
# ======================================================

DATA_YAML = "/kaggle/input/datasets/sarthaksaitwal/road-defect-dataset/data/data.yaml"
RUNS_DIR = OUTPUT_DIR

MODEL_SIZE = "s"
MODEL_FILE = f"yolov8{MODEL_SIZE}.pt"
RUN_NAME = f"yolov8{MODEL_SIZE}_road_defects"

# ======================================================
# DAGSHUB SETUP (NEW REPO)
# ======================================================

dagshub.init(
    repo_owner="sarthaksaitwal3",
    repo_name="final-road-detect-defect",   # ✅ NEW REPO
    mlflow=True,
)

mlflow.set_experiment("YOLO_model")  # ✅ NEW EXPERIMENT

print("Checking dataset path...")
print("DATA_YAML:", DATA_YAML)
print("Exists?", os.path.exists(DATA_YAML))

# ======================================================
# TRAINING CONFIG
# ======================================================

training_config = {
    "data": DATA_YAML,
    "epochs": 70,
    "resume": False,
    "imgsz": 768,
    "batch": 16 if torch.cuda.is_available() else 8,
    "device": device,
    "patience": 15,
    "optimizer": "SGD",
    "lr0": 0.01,
    "momentum": 0.937,
    "weight_decay": 0.0005,
    "warmup_epochs": 3,
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 10,
    "translate": 0.1,
    "scale": 0.5,
    "flipud": 0.0,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "cos_lr": True,
    "close_mosaic": 10,
    "save": True,
    "val": True,
    "plots": True,
    "workers": 8,
    "cache": "disk" if IN_KAGGLE else True,
}

# ======================================================
# TRAINING
# ======================================================

torch.cuda.empty_cache()
from ultralytics import settings
settings.update({"mlflow": False})

model = YOLO(MODEL_FILE)

with mlflow.start_run(run_name=RUN_NAME):

    # Log params
    mlflow.log_param("model_size", MODEL_SIZE)
    mlflow.log_param("epochs", training_config["epochs"])
    mlflow.log_param("imgsz", training_config["imgsz"])
    mlflow.log_param("batch_size", training_config["batch"])
    mlflow.log_param("optimizer", training_config["optimizer"])
    mlflow.set_tag("architecture", f"YOLOv8{MODEL_SIZE}")
    mlflow.log_param("device", str(device))
    mlflow.log_param("environment", "Kaggle" if IN_KAGGLE else ("Colab" if IN_COLAB else "Local"))

    try:
        git_commit = subprocess.check_output(["git", "rev-parse", "HEAD"]).strip().decode()
        mlflow.log_param("git_commit", git_commit)
    except Exception:
        pass

    print("\n🚀 Training Started...\n")

    results = model.train(
        project=RUNS_DIR,
        name=RUN_NAME,
        exist_ok=True,
        **training_config,
    )

    print("\n✅ Training Completed!")

    # ======================================================
    # LOAD BEST MODEL
    # ======================================================

    best_model_path = f"{RUNS_DIR}/{RUN_NAME}/weights/best.pt"
    best_model = YOLO(best_model_path)

    metrics = best_model.val(data=DATA_YAML)

    precision = float(metrics.box.mp)
    recall = float(metrics.box.mr)
    map50 = float(metrics.box.map50)
    map5095 = float(metrics.box.map)

    f1 = (2 * (precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0

    # Log metrics
    mlflow.log_metric("mAP50", map50)
    mlflow.log_metric("mAP50-95", map5095)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # ======================================================
    # LOG MODEL + ARTIFACTS
    # ======================================================

    # 1️⃣ Log raw best.pt
    mlflow.log_artifact(best_model_path, artifact_path="artifacts")

    # 2️⃣ Log structured MLflow model
    mlflow.pytorch.log_model(
        pytorch_model=best_model.model,
        name="model"
    )

    print("✅ Model + artifacts logged successfully")

    print("\n📊 FINAL METRICS")
    print("-" * 50)
    print(f"mAP50:     {map50:.3f}")
    print(f"mAP50-95:  {map5095:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall:    {recall:.3f}")
    print(f"F1 Score:  {f1:.3f}")
    print("-" * 50)

torch.cuda.empty_cache()

print("\n🎉 Done!")
print("Check Experiments → YOLO_model in DagsHub.")